In [1]:
#byte pair encoding
import tiktoken 
import pandas as pd
from importlib.metadata import version
print("tiktoken version:", version("tiktoken"))

#tiktoken: python library that implements the byte pair encoding algorithm
#breaks down unknown words into knownsubwords/characters 

tokenizer = tiktoken.get_encoding("gpt2") 
text = ("Hello do you like tea? <|endoftext|> In the sunlit terraces" "of some unknown place")
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

tiktoken version: 0.9.0
[15496, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 6439, 1295]


In [2]:
#convert integers (tokenized input) back into the original input
strings = tokenizer.decode(integers)
print(strings)


Hello do you like tea? <|endoftext|> In the sunlit terracesof some unknown place


In [4]:
#input target pairs using sliding window method
#tokenize the whole "the-verdict.txt"

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))
#length is 5145
#only take everything after 50
enc_sample = enc_text[50:]
context_size = 4 #how many tokens are included in the input
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

#print out the next word prediction
for i in range(1, context_size+1):
    context = enc_sample[:i]
    output = enc_sample[i]
    print(context, "----->", output)
    print(tokenizer.decode(context), "---->", tokenizer.decode([output]))


#these need to be tensors (multidimensional arrays): input tensor and target ouptut tensor

5145
x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]
[290] -----> 4920
 and ---->  established
[290, 4920] -----> 2241
 and established ---->  himself
[290, 4920, 2241] -----> 287
 and established himself ---->  in
[290, 4920, 2241, 287] -----> 257
 and established himself in ---->  a


In [5]:
#converting into input and output tensors
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDataset(Dataset):
    """An abstract class representing a :class:`Dataset`.

    All datasets that represent a map from keys to data samples should subclass
    it. All subclasses should overwrite :meth:`__getitem__`, supporting fetching a
    data sample for a given key. Subclasses could also optionally overwrite
    :meth:`__len__`, which is expected to return the size of the dataset by many
    :class:`~torch.utils.data.Sampler` implementations and the default options
    of :class:`~torch.utils.data.DataLoader`. Subclasses could also
    optionally implement :meth:`__getitems__`, for speedup batched samples
    loading. This method accepts list of indices of samples of batch and returns
    list of samples.

    .. note::
      :class:`~torch.utils.data.DataLoader` by default constructs an index
      sampler that yields integral indices.  To make it work with a map-style
      dataset with non-integral indices/keys, a custom sampler must be provided. """
    def __init__(self, txt, tokenizer, max_length, stride):
        '''
        max_length = number of tokens per input sequence
        stride = how much the input sequence shifts
        '''
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride): #range(start, stop, step)
            #start at 0, stop at length of whole text, iterate with i*stride
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __getitem__(self, idx): 
        #for fetching a data sample for a given key
        #returns single row for dataset
        return self.input_ids[idx], self.target_ids[idx]
    
    def __len__(self):
        #returns the total number of rows in the dataset
        return len(self.input_ids)

In [6]:
#a dataloader using theGPTDataset class to generate a dataset
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    '''
    dataset (Dataset): dataset from which to load the data.
        batch_size (int, optional): how many samples per batch to load, default 1
        shuffle (bool, optional): data reshuffled at every epoch, default False
        num_workers (int, optional): how many subprocesses to use for data
            loading. ``0`` means that the data will be loaded in the main process.
            (default: ``0``)
            #number of CPU processes to use for preprocessing
        drop_last (bool, optional): Drop last batch if smaller than batch_size, default False
    '''
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDataset(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset=dataset, batch_size = batch_size, shuffle=shuffle, drop_last=drop_last,num_workers=num_workers
    )
    return dataloader



In [7]:
#testing dataloader with batch size of 1 for an LLM with a context size of 4 to develop an intuition of how GPTDataclassvV1 and create_dataloader_v1 function work together
#small batch sizes require less memory during training but lead to more noisy model updates
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

#stride=1, so second batch will have step size of 1
second_batch = next(data_iter)
print(second_batch)


[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [8]:
#lets look at an example with batch size = 8 and max length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs: ", inputs)
print("outputs: ", targets)

Inputs:  tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
outputs:  tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [9]:
#convert these token IDs into embedding vectors
#random embedding weight matrix (length of total vocab) by (dimension of each word/token)
#each token ID represents an index in the weight matrix which represents a row of the dimension
#example
input_ids = torch.tensor([2, 3, 5, 1])
#instantiate an embedding layer in pytorch
vocab_size = 6 #instead of 50,275 words in the bype pair encoding tokenizer vocabulary
output_dim = 3 #each word has a row of output dim, gpt 3 has embedding size of 12,288 dimensions

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight) #weight matrix of small random values, which will be optimized during training

print(embedding_layer(torch.tensor([3]))) #will be row 4 of the embedding_layer

#apply it to input_ids so that it can have a corresponding weight matric
print(embedding_layer(input_ids))

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)
tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [ ]:
#problem: no order specified for the token ID
#solution: create positional embeddings we add to the tokenIDs
